In [1]:
import requests
import json
import pandas as pd
import itertools
from urllib import parse
import time
import numpy as np
import pickle

In [2]:
# 크롤링 우회
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))
print(generate_user_agent(os=('mac', 'linux'), device_type='desktop'))

navigator = generate_navigator()
print(navigator)
print(navigator['platform'])

Mozilla/5.0 (Windows NT 5.1; WOW64; Trident/7.0; rv:11.0) like Gecko
Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36
{'os_id': 'win', 'navigator_id': 'chrome', 'platform': 'Windows NT 6.1; Win64; x64', 'oscpu': 'Windows NT 6.1; Win64; x64', 'build_version': '86.0.4240.183', 'build_id': None, 'app_version': '5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36', 'app_name': 'Netscape', 'app_code_name': 'Mozilla', 'product': 'Gecko', 'product_sub': '20030107', 'vendor': 'Google Inc.', 'vendor_sub': '', 'user_agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'}
Windows NT 6.1; Win64; x64


In [3]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time 
import requests
import os
from urllib import parse

In [4]:
import pymysql

# MySQL 연결 정보 설정
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234', db='dun', charset='utf8')

cur = conn.cursor()


In [5]:
남귀검사 = ['眞 웨펀마스터', "眞 버서커", "眞 소울브링어", "眞 아수라", "眞 검귀"]

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from urllib.parse import quote

# 웹 드라이버 시작
driver = webdriver.Chrome()

남귀검사 = ['眞 웨펀마스터', "眞 버서커", "眞 소울브링어", "眞 아수라", "眞 검귀"]

name_list = []
server_list = []
damage_list = []
val_list = []
job_list = []

for j in 남귀검사:
    for i in range(1, 101):
        url = "https://dundam.xyz/damage_ranking?page={}&type=7&job={}&baseJob=%EA%B7%80%EA%B2%80%EC%82%AC(%EB%82%A8)&weaponType=%EC%A0%84%EC%B2%B4&weaponDetail=%EC%A0%84%EC%B2%B4"
        driver.get(url.format(i, quote(j)))
        time.sleep(2)


        if i == 1:
            val_elements = driver.find_elements(By.CLASS_NAME, 'val')
            server_elements = driver.find_elements(By.CLASS_NAME, 'svname')

            val_list += [element.text for element in val_elements[8:]]
            server_list += [element.text for element in server_elements[8:]]

            name_list += [x.text for x in driver.find_elements(By.CLASS_NAME, 'nik')]
            damage_list += [j.text for j in driver.find_elements(By.CLASS_NAME, 'damegen')]
            name_list = " ".join(name_list).split()
            damage_list = " ".join(damage_list).split()

        else:
            name_list += [x.text for x in driver.find_elements(By.CLASS_NAME, 'nik')]
            server_list += [k.text for k in driver.find_elements(By.CLASS_NAME, 'svname')]
            damage_list += [j.text for j in driver.find_elements(By.CLASS_NAME, 'damegen')]
            val_list += [y.text for y in driver.find_elements(By.CLASS_NAME, 'val')]

            name_list = " ".join(name_list).split()
            server_list = " ".join(server_list).split()
            damage_list = " ".join(damage_list).split()
            val_list = " ".join(val_list).split()

        job_list += [j] * 10

# 웹 드라이버 종료
driver.quit()


The chromedriver version (116.0.5845.96) detected in PATH at c:\myproject\dunfa\chromedriver.exe might not be compatible with the detected chrome version (117.0.5938.150); currently, chromedriver 117.0.5938.149 is recommended for chrome 117.*, so it is advised to delete the driver in PATH and retry


In [25]:
data = {
    'name' : name_list,
    'job' : job_list,
    'server' : server_list,
    'val' : val_list,
    'damage' : damage_list
}
df = pd.DataFrame(data)
df

,name,job,server,val,damage
0,문일성,眞 웨펀마스터,카인,57796,"474,726,087,186"
1,빨래줄,眞 웨펀마스터,카인,55720,"356,575,345,332"
2,마우스,眞 웨펀마스터,시로코,55489,"354,525,993,007"
3,젠틀한검신,眞 웨펀마스터,카인,55874,"341,692,645,347"
4,lnvincible,眞 웨펀마스터,카인,54989,"336,648,376,689"
...,...,...,...,...,...
4995,천퍼센트,眞 검귀,시로코,51593,"117,384,798,274"
4996,프로게이G,眞 검귀,카인,51010,"117,379,679,948"
4997,鬼바리,眞 검귀,안톤,50724,"117,356,763,906"
4998,하늘검귀,眞 검귀,디레지에,51617,"117,335,535,403"


In [26]:
for name, job, server, damage, val in zip(name_list, job_list, server_list, damage_list, val_list):
    insert_query = """
        INSERT INTO dundam_user (name, job, server, damage, val)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        job = VALUES(job), server = VALUES(server), damage = VALUES(damage), val = VALUES(val)
    """
    
    cur.execute(insert_query, (name, job, server, damage, val))
    
conn.commit()


In [253]:
server = []
for value in server_list :
    if value == '카인' : 
        server.append('cain')
    elif value == '디레지에' :
        server.append('diregie')
    elif value == '시로코' : 
        server.append('siroco')
    elif value == '프레이' : 
        server.append('prey')
    elif value == '카시야스' :
        server.append('casillas')
    elif value == '힐더' : 
        server.append('hilder')
    elif value == '안톤' : 
        server.append('anton')
    elif value == '바칼' : 
        server.append('bakal')

In [251]:
name_list.__len__()

5000

In [254]:
server.__len__()

5000

In [256]:
for i, j in list(zip(server, name_list)) : 
    print(i)

cain
cain
siroco
cain
cain
prey
cain
cain
cain
anton
cain
cain
diregie
anton
siroco
hilder
cain
casillas
diregie
cain
siroco
cain
anton
cain
cain
casillas
cain
casillas
diregie
casillas
prey
cain
prey
cain
siroco
hilder
bakal
cain
hilder
bakal
siroco
anton
bakal
prey
prey
prey
bakal
cain
bakal
casillas
cain
prey
prey
cain
cain
cain
siroco
hilder
prey
casillas
cain
bakal
cain
siroco
casillas
prey
diregie
bakal
bakal
casillas
bakal
cain
anton
hilder
hilder
casillas
casillas
diregie
anton
cain
cain
cain
prey
cain
casillas
bakal
diregie
cain
casillas
diregie
casillas
cain
hilder
prey
anton
anton
casillas
prey
cain
prey
cain
anton
prey
cain
cain
bakal
prey
diregie
cain
diregie
anton
hilder
siroco
cain
diregie
diregie
cain
cain
cain
bakal
cain
cain
casillas
casillas
hilder
cain
cain
casillas
cain
cain
cain
siroco
hilder
cain
diregie
prey
diregie
diregie
cain
hilder
cain
cain
siroco
bakal
casillas
bakal
cain
cain
cain
prey
diregie
cain
siroco
cain
hilder
casillas
cain
casillas
bakal
casillas


In [257]:
answer = []

with requests.Session() as session:
    for server, name in list(zip(server, name_list)) :
        # URL 조합
        url = "https://api.neople.co.kr/df/servers/{}/characters?characterName={}&apikey=kNmTp2mebpjBykLw8aubHUYRWVm8XHjN".format(server, quote(name))
        req = session.get(url)
        # API 응답의 결과를 확인하여 유효한 캐릭터 ID가 있는 경우에만 answer 리스트에 추가
        answer.append(req.json())

In [259]:
answer.__len__()

5000

In [260]:
import pickle

# 파일로 저장
with open('my_dict.pkl', 'wb') as f:
    pickle.dump(answer, f)

In [5]:
# 파일 불러오기
with open('my_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [7]:
answer = loaded_dict.copy()

In [262]:
answer[0]['rows'][0]

{'serverId': 'cain',
 'characterId': 'd417c0189ebc9fcf3936a99db993872a',
 'characterName': '문일성',
 'level': 110,
 'jobId': '41f1cdc2ff58bb5fdc287be0db2a8df3',
 'jobGrowId': '37495b941da3b1661bc900e68ef3b2c6',
 'jobName': '귀검사(남)',
 'jobGrowName': '眞 웨펀마스터',
 'fame': 57796}

In [289]:
for i in answer:
    if 'rows' in i and i['rows'] and len(i['rows']) > 0:
        serverId = i['rows'][0]['serverId']
        characterId = i['rows'][0]['characterId']
        characterName = i['rows'][0]['characterName']
        level = i['rows'][0]['level']
        jobId = i['rows'][0]['jobId']
        jobGrowId = i['rows'][0]['jobGrowId']
        jobName = i['rows'][0]['jobName']
        jobGrowName = i['rows'][0]['jobGrowName']
        fame = i['rows'][0]['fame']
        
        insert_query2 = """
            INSERT INTO neople_user (id, server, name, level, jobid, growid, jobname, growjob, fame)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        cur.execute(insert_query2, (characterId, serverId, characterName, level, jobId, jobGrowId, jobName, jobGrowName, fame))

conn.commit()


In [287]:
answer[0]['rows'][0]['serverId']

'cain'

In [290]:
url = "https://api.neople.co.kr/df/servers/cain/characters/d417c0189ebc9fcf3936a99db993872a/equip/equipment?apikey=kNmTp2mebpjBykLw8aubHUYRWVm8XHjN"
req = requests.get(url)
# API 응답의 결과를 확인하여 유효한 캐릭터 ID가 있는 경우에만 answer 리스트에 추가
req.json()

{'characterId': 'd417c0189ebc9fcf3936a99db993872a',
 'characterName': '문일성',
 'level': 110,
 'jobId': '41f1cdc2ff58bb5fdc287be0db2a8df3',
 'jobGrowId': '37495b941da3b1661bc900e68ef3b2c6',
 'jobName': '귀검사(남)',
 'jobGrowName': '眞 웨펀마스터',
 'adventureName': '문씨조선',
 'guildId': '1342ebaed4120b98269d6d3a98af01e2',
 'guildName': '깨굴',
 'equipment': [{'slotId': 'WEAPON',
   'slotName': '무기',
   'itemId': '0469d88bf1362c05c1b3ad70c013367c',
   'itemName': '불가침의 영역 - 광검',
   'itemTypeId': '4ffb6f14b86f5c818a925bf58022686e',
   'itemType': '무기',
   'itemTypeDetailId': 'bd9c897afd0825ad854f0a81fc8d42ae',
   'itemTypeDetail': '광검',
   'itemAvailableLevel': 105,
   'itemRarity': '레전더리',
   'setItemId': None,
   'setItemName': None,
   'skin': {'itemId': '98ee63e8e9d32d05260aed5a16523a6f',
    'itemName': '마법의 견습 광검',
    'itemRarity': '레어'},
   'reinforce': 13,
   'itemGradeName': '최상급',
   'enchant': {'status': [{'name': '모든 속성 강화', 'value': 15},
     {'name': '물리 공격력', 'value': 30},
     {'name':

In [16]:
sql = "select id from neople_user"
# SQL 실행
result = pd.read_sql(sql, conn)
result

C:\Users\nbac4\AppData\Local\Temp\ipykernel_10716\728886612.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(sql, conn)


,id
0,d417c0189ebc9fcf3936a99db993872a
1,78c3a52057f580c290e5d58952592598
2,8799a97922e6564081b5ee52bbe96e9f
3,1ea43cb9e5cee95ae042ae8e5efa3784
4,b800b67866b83e497a473962e14e9f53
...,...
6246,0b6fa7569d36791889da18c410ce501c
6247,18d4f35be991819e248142cfd5d739ed
6248,bed09fd993c8868e5de798ecda620b9b
6249,21d9f187a4175584e7ec44f84e368b5c


d417c0189ebc9fcf3936a99db993872a
78c3a52057f580c290e5d58952592598
8799a97922e6564081b5ee52bbe96e9f
1ea43cb9e5cee95ae042ae8e5efa3784
b800b67866b83e497a473962e14e9f53
3779cbc5440c60474e2b9d73e3a65a1a
5e7eaf266dae15f2eb7ca301c0fa269e
108ca525957edbaf5634c82ef2749ff7
dcc20f5e87cb3da7a819fde2d003b459
a2020b9a7d048b6b07f6cd86fce5a5c4
e270463952c8253766273f4a8ae83cc3
377eb7a7f9f29bb5b5bb5878b4a52c0d
d9d26cc5db8344d1fc8aa3df4ab324a5
608621971ad2e8863a66a5698be18658
a304a10da75a04af341215e5da1b9a72
026c586858dfdad89f06396c332157f2
3cb7ab65900977ad2865a2c1cb4c4066
4a43bd715d3b3da937def9d7cdfe3da4
7ee3df2982920d6d98f620cf3819daff
3e99009c845592672efb82282abebe69
06916ee68e7fef22af2b6e2bced04c6a
9dcb04cff15c71bd13ca109b84016032
e313492a0e68b168062da39134403a75
24237d3fc99a633c6b5cb73e977f2641
43f28f0e814b35ae38b5229e62d671bb
884c9a73a3951ff32b5acc8e93e0a0e5
b570779b018b1e8ac6d4ea06a6dfc204
28b557efac046faa68699b7b7ec8f451
f72da333c4b68abf3784e418a2da9bbc
11db969d97df8fbcaf24b60802fd25bd
c4d1d90609

In [ ]:
url = "https://api.neople.co.kr/df/servers/{}/characters/{}/equip/equipment?apikey=kNmTp2mebpjBykLw8aubHUYRWVm8XHjN"
req = requests.get(url)
# API 응답의 결과를 확인하여 유효한 캐릭터 ID가 있는 경우에만 answer 리스트에 추가
req.json()

In [291]:
test = req.json()

In [316]:
test['characterId']
test['characterName']
test['equipment'][0]['slotId']
test['equipment'][0]['slotName']
test['equipment'][0]['itemName']
test['equipment'][0]['itemTypeDetail']
test['equipment'][0]['itemAvailableLevel']
test['equipment'][0]['itemRarity']
test['equipment'][0]['reinforce']
test['equipment'][0]['enchant']['status'][0]['name']
test['equipment'][0]['enchant']['status'][0]['value']
test['equipment'][0]['enchant']['status'][1]['name']
test['equipment'][0]['enchant']['status'][1]['value']
test['equipment'][0]['enchant']['status'][2]['name']
test['equipment'][0]['enchant']['status'][2]['value']
test['equipment'][0]['enchant']['status'][3]['name']
test['equipment'][0]['enchant']['status'][3]['value']
test['equipment'][0]['bakalInfo']['options'][0]['explain']
test['equipment'][0]['bakalInfo']['options'][1]['explain']
test['equipment'][0]['bakalInfo']['options'][2]['explain']
test['equipment'][0]['customOption']['damage']
test['equipment'][0]['customOption']['level']
test['equipment'][0]['customOption']['options'][0]['explain']
test['equipment'][0]['customOption']['options'][1]['explain']
test['equipment'][0]['customOption']['options'][2]['explain']
test['equipment'][0]['customOption']['options'][3]['explain']

'스킬 퀵슬롯의 빈 슬롯 1개마다 스킬 공격력 2% 증가 (최대 6중첩)'